# Testing plots 

In [ ]:
import pandas as pd
import numpy as np
import pylab as plt

# usually, two decimals suffice for displaying DataFrames (NB internally, precision may be higher)
pd.options.display.precision = 2

import sys
sys.path.append('../data/')

%load_ext autoreload


%matplotlib widget

from preprocessor import Preprocessor

In [ ]:
%%time
# Prerequisite: for this example to work, you need to have the b4b_raw_properties.parquet, located e.g. in the ../data/ folder.
# One way to get this is to run B4BExtractionBackup.ipynb first
df = pd.read_parquet('b4b_raw_properties.parquet', engine='pyarrow')

#sorting the DataFrame index is needed to get good performance on certain filters
if not df.index.is_monotonic_increasing:
  df = df.sort_index()  

In [ ]:
def get_unit(prop:str) -> str:
    return prop.split('__')[-1]

In [ ]:
# This cell can be used to plot one or more properties in one or more rooms for one or more sources
# TODO: unstack sources and put data from different sources with same unit in same subplot, while still clearly identifying source in label and color 

for id, source in list(df.index.droplevel(2).to_flat_index().unique()):
    props_with_data = [prop for prop in list(df.columns) if df.loc[id,source][prop].count()>0] 
    units_with_data = np.unique(np.array([get_unit(prop) for prop in props_with_data]))
    unit_tuples = [tuple([prop for prop in props_with_data if get_unit(prop) == unit]) for unit in units_with_data]
    df.loc[id,source][props_with_data].plot(
        subplots = unit_tuples,
        style='.--',
        title=f'room: {id}, source: {source}'
    )